In [ ]:
'''
talib
'''

In [ ]:
import talib
import numpy as np

In [ ]:
high = np.array([1.0, 2.0, 3.0, 4.0, 5.0, 6.0])
low = np.array([6.0, 5.0, 4.0, 3.0, 2.0, 1.0])
close = np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0])
a = talib.ATR(high, low, close, 2)
a

In [ ]:
'''
tushare
'''

In [2]:
import tushare as ts
from datetime import datetime, timedelta

In [5]:
import tushare as ts

In [16]:
cons = ts.get_apis()

In [12]:
df = ts.tick('rb1805', conn = cons, date = '2017-12-01', asset = 'X') 

In [ ]:
df

In [12]:
a = df.ix[1]
the = a['date']
dt = the.to_datetime()
print dt
if dt.time() >= datetime.time(21, 0):
    dt = dt - timedelta(1)

print dt

2017-09-21 21:00:00


TypeError: descriptor 'time' requires a 'datetime.datetime' object but received a 'int'

In [ ]:
for index, row in df.iterrows():
    print row['date'], ' '*2, row['vol']

In [26]:
a = {'a':1}
b = {'b':2}
c = a+b
print c

TypeError: unsupported operand type(s) for +: 'dict' and 'dict'

In [37]:
a = [1,3,2]
b = [1,2,3]
c = a+b
print c
c.remove(2)
c.remove(2)
print c

[1, 3, 2, 1, 2, 3]
[1, 3, 1, 3]


[1, 2]